In [ ]:
#default_exp surprise.processing
%load_ext autoreload
%autoreload 2

# surprise.processing
> Processing the different stream of data to calculate responses of retinal cells

In [ ]:
#export
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import cluster
import scipy.ndimage as ndimage
import scipy.signal as signal
import scipy as sp
from cmath import *
import itertools
import random

from theonerig.core import *
from theonerig.utils import *
from theonerig.modelling import *

# Surprise function for data non-linear transformation

In [ ]:
#export
def prior_p_m_num(window,seq):   
    ''' it calculates the numerator of the prior'''
    ms=seq.size
    counter=0
    for i in range(window.size+1):
        if i>=ms:
            comp=window[i-ms:i]==seq
            if comp.all():
                counter+=1
    return counter

def prior_p_m_denum(window,seq):   #window is size w and seq is size ms
    ''' it calculates the denumerator of the prior'''
    return window.size-seq.size-1

def p_d(window, D):
    ''' it calculates the evidence'''
    return window[window==D].size/len(window)

def p_d_m(window,seq, D):  
    ''' it calculates the likelihood'''
    ms=seq.size
    count=0
    for i in np.where(window==D)[0]:
        if i-ms>=0:
            comp=window[i-ms:i]==seq
            if comp.all():
                count+=1
    return count/prior_p_m_num(window,seq)

def summing_function(window,D,ms): 
    '''it finds all the possible stataes sequences to sum across'''
    Ms=[]
    for j in np.where(window==D)[0]:
        if len(window[j-ms:j])==ms:
            Ms.append(window[j-ms:j])
    #return np.unique(np.array(Ms),axis=0)
    return np.array(Ms)


def surprise(array, window_lenght, ms):
    '''changes the trace of a pixel values according to how surprising they are'''
    filtered_array=np.zeros((len(array)))
    for time, value in enumerate(array):
        if time<window_lenght:
            filtered_array[time]=value
        else:
            Ms=summing_function(array[time-window_lenght:time],value, ms) 
            if Ms.size ==0:
                surprise=1
            else:
                Ms=np.unique(Ms,axis=0)
                surprise=0
                for M in Ms:
                    likelihood=p_d_m(array[time-window_lenght:time],M, value)
                    p_m=prior_p_m_num(array[time-window_lenght:time],M)/prior_p_m_denum(array[time-window_lenght:time],M)
                    posterior=likelihood*p_m/p_d(array[time-window_lenght:time],value)
                    surprise+=posterior*np.log(posterior/p_m)
            filtered_array[time]=surprise
    return filtered_array

In [ ]:
#export

def mixed_stim_finder(stimulus,reps, ampl):
    '''a function that finds out the time instants where a series of given reps of given amplitude ends. The given number 
    is the time at which there is the onset of the last flash, so 5 frames later there is the missing stim'''
    times=[]
    i=np.where(stimulus !=255)[0][0]
    j=0
    while i<len(stimulus):
        if list(np.where(stimulus[i:] !=255)[0]):
            i=np.where(stimulus[i:] !=255)[0][0]+i
        if stimulus[i]==ampl:
            j+=1
            if j==reps:
                if stimulus[i+5]!=ampl:
                    times.append(i)
        else:
            j=0
        i+=4
    return np.array(times)

def altern_stim_finder(stimulus,reps):
    '''the function finds where a series of a given number of douplets AB of stimuli ends with a surprising duplet AA.
    The given time is the time at which the first of the two As of the surprising duplet onsets. The surprise one is than
    expected not before 5 frames after the reported time'''
    times=[]
    i=np.where(stimulus !=255)[0][0]
    j=0
    while i<len(stimulus)-6:
        #if list(np.where(stimulus[i-2:] !=255)[0]):
            #i=np.where(stimulus[i:] !=255)[0][0]+i
        #print(i)
        i+=5    
        if stimulus[i]==170:
            j+=1
        elif j==reps and stimulus[i]==0:
            times.append(i-5)
            j=0
        elif stimulus[i]==0:
            j=0
        i+=5
        #print(j)
        if reps==16:
            if i>14431:
                break
    return np.array(times)

def tripl_stim_finder(stimulus):
    '''the function finds where a series of 16 reps of triplets ABC is interrupted by a different triplet ABB that differs by 
    the previous 16 for the last flash of the triplet. The given instant is the one where the second flash of the surprising 
    triplet onsets for consistency with the other finders. In this way infact the surprise response is expected not sooner than 
    5 frames after the reported time.
    '''
    times=[]
    i=np.where(stimulus[1:] !=255)[0][0]
    while i<len(stimulus)-6 and len(times)<240:
        if list(np.where(stimulus[i-2:] !=255)[0]):
            i=np.where(stimulus[i:] !=255)[0][0]+i
        #print(i)
        i+=270
        if list(np.where(stimulus[i-2:] !=255)[0]):
            i=np.where(stimulus[i:] !=255)[0][0]+i
            times.append(i-40)
        #print(i)
        else:
            times.append(np.where(stimulus[:i] !=255)[0][-1]-2)
            
    return np.array(times)

def finder_of_both(stimulus,half): 
    '''finds where are the AAB and the AAC either in the first or in the second half of stimulation.
    for the starting and ending fixed repetitions there are separate functions
    '''
    timesB=[]
    timesC=[]
    if half==1:
        st=C_stim_finder(stimulus)[-1]
    if half==2:
        st=18120
    i=np.where(stimulus[st:st+100])[0][0]+st
    j=0
    while i>=st and j<1000:

        if list(np.where(stimulus[i-2:i+100] !=255)[0]):
            i=np.where(stimulus[i:] !=255)[0][0]+i
        #print(i)
        i+=14
        if list(np.where(stimulus[i-2:i+15] ==170)[0]):
            i=np.where(stimulus[i:i+15] ==170)[0][0]+i
            timesB.append(i)
            j+=1
        elif list(np.where(stimulus[i-2:i+15] ==85)[0]):
            i=np.where(stimulus[i:i+15] ==85)[0][0]+i
            timesC.append(i)
            j+=1
        else:
            print("No matches!")      
    return np.array(timesB), np.array(timesC)

def psth(num,function,stimulus, resp, cell, reps,ampl=None):
    '''function that plots a raster plot and consequent psth over trials for a given series of stimuli.
    num=# of different raster that I generate in the notebook
    function= different finder from the above depending on which type of stimulus I am analyzing
    stimulus= the presented stimulus
    cell=# the ID of the neuron
    reps=# of reps of flashes before the missing one
    ampl=amplitude of the flases. Needed only for mixed stim.
    resp= the response to the stimulus
    '''
    if ampl != None:
        globals()["times"+str(num)]=function(stimulus,reps,ampl)
    else:
        globals()["times"+str(num)]=function(stimulus,reps)
    
    print(globals()["times"+str(num)].shape)

    rows=len(globals()["times"+str(num)])
    window=20
    globals()["data"+str(num)]=np.zeros((rows, window*3))
    for i,k in enumerate(globals()["times"+str(num)]):
        globals()["data"+str(num)][i]=resp[k+5-2*window:k+5+window,cell]
    plt.figure()
    plt.imshow(globals()["data"+str(num)])
    plt.figure()    
    plt.plot(np.sum(globals()["data"+str(num)], axis=0))

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 02_processing.ipynb.
Converted 03_modelling.ipynb.
Converted 04_plotting.ipynb.
Converted 05_database.ipynb.
Converted 10_synchro.io.ipynb.
Converted 11_synchro.extracting.ipynb.
Converted 12_synchro.processing.ipynb.
Converted 13_leddome.ipynb.
Converted 20_surprise.processing.ipynb.
Converted 99_testdata.ipynb.
Converted index.ipynb.
